# Exploración inicial

Este Notebook realiza una comparación inicial entre los subtítulos de las películas de inmigración y una muestra de los subtítulos de las películas de no inmigración basada en la construcción de dos puntajes de la cantidad de palabras asociadas a la inmigración presentes en las películas.

In [ ]:
# Importamos librerías 

## Módulos generales
from libraries import *

In [ ]:
# Completar con directorios
gitwd = ""
datawd = ""

## Importar subtítulos 

In [ ]:
films = pd.read_pickle(datawd + "/films_sample_inputdf.pkl")     
films.just_migra.value_counts()

## Importar un conjunto de palabras asociadas a la inmigración junto con puntajes de asociación, y obtener _stems_ únicos 

In [ ]:
# Usamos los Thesaurus de Sketch Engine
os.chdir(datawd+"/immigration corpus/sketchengine")
files = os.listdir()

thesaurusf   = [f for f in files if 'thesaurus' in f]
thesaurus = pd.DataFrame()

# importamos todos los datos
for f in range(len(thesaurusf)):
    file = thesaurusf[f]
    df_keyword = pd.read_csv(file, nrows = 1, skiprows = 1)
    df = pd.read_csv(file, skiprows = 2)
    df.columns = ["word", "similarity", "freq"]
    keyword = df_keyword.index[0]
    df['keyword'] = keyword
    if keyword in ["immigrant", "immigration", "migrant", "migration"]:
        print(keyword, file, df.shape)
        thesaurus = pd.concat([thesaurus, df]).reset_index(drop = True)

# Stemming
thesaurus["word_porter"] = [PorterStemmer().stem(x) for x in thesaurus.word]
print(thesaurus.shape)

# Agrupar por stem y calcular la similaridad máxima con las palabras clave
thesaurus = thesaurus.groupby('word_porter', as_index = False).agg({'similarity': 'max', 
                                                                    'freq': 'mean',
                                                                    'keyword': 'unique'})

thesaurus['n_keywords'] = [len(k) for k in thesaurus.keyword]

thesaurus = thesaurus.sort_values(['similarity', 'n_keywords'],ascending = False).reset_index(drop = True)
print(thesaurus.shape)

In [ ]:
thesaurus.head(10)

## Puntajes de contenido de inmigración por película

- Puntaje I: para cada película, sumar la similaridad de cada palabra que aparece, sin importar cuántas veces aparece
- Puntaje II: para cada película, sumer la similaridad de cada palabra por cada vez que aparece en el subtítulo

In [ ]:
def score1(text):
    score = np.sum([thesaurus.similarity[w] if thesaurus.word_porter[w] in text
                    else 0 
                    for w in range(thesaurus.shape[0]) ])
    return score

def score2(text):
     
    score = np.sum([thesaurus.similarity[w] * 
                    len(re.findall(thesaurus.word_porter[w], text)) 
                    for w in range(thesaurus.shape[0]) ])
    
    return score

films["score1"] = [score1(films.s[i]) for i in tqdm(range(films.shape[0]))]
films["score2"] = [score2(films.s[i]) for i in tqdm(range(films.shape[0]))]
films["len_text"] = [len(films.s[i]) for i in tqdm(range(films.shape[0]))] # también medimos el largo del texto

## Comparación

In [ ]:
# Medidas resumen PUNTAJE 1
print("INMIGRACIÓN")
print(films[films.just_migra == 1].score1.describe())
print("--------")
print("NO-INMIGRACIÓN")
print(films[films.just_migra == 0].score1.describe())

In [ ]:
# Histogramas superpuestos PUNTAJE 1
bins = np.linspace(0, 200, 20)
plt.figure(figsize=(8, 6), dpi=80)
plt.hist(films[films.just_migra == 0].score1, bins, alpha=0.5, label='Películas No Inmigración', density =True)
plt.hist(films[films.just_migra == 1].score1, bins, alpha=0.5, label='Películas Inmigración', density = True)
plt.legend(loc='upper left')
plt.savefig(datawd + f"/exploration_hist_score1.png", dpi = 300)
plt.show()

In [ ]:
# Medidas resumen PUNTAJE 2
print("INMIGRACIÓN")
print(films[films.just_migra == 1].score2.describe())
print("--------")
print("NO-INMIGRACIÓN")
print(films[films.just_migra == 0].score2.describe())


In [ ]:
# Histogramas superpuestos PUNTAJE 2
plt.figure(figsize=(8, 6), dpi=80)
plt.hist(films[films.just_migra == 0].score2, bins = 20, alpha=0.5, label='Películas No Inmigración', density =True)
plt.hist(films[films.just_migra == 1].score2, bins = 20, alpha=0.5, label='Películas Inmigración', density = True)
plt.legend(loc='upper left')
plt.savefig(datawd + f"/exploration_hist_score2.png", dpi = 300)
plt.show()